In [3]:
import random
import numpy as np
from typing import List, Tuple, Set
from collections import namedtuple
from sklearn.metrics import jaccard_score
import pandas as pd
import itertools # Import itertools

# Define structure for nodes
Node = namedtuple('Node', ['id', 'profit', 'time_window'])

# Simulated nodes with id, profit, and time window
nodes = [
    Node(0, 0, (0, 100)),
    Node(1, 10, (5, 50)),
    Node(2, 15, (10, 60)),
    Node(3, 20, (0, 40)),
    Node(4, 25, (20, 70)),
    Node(5, 18, (30, 90))
]

# Simulated travel time matrix
travel_times = {
    (i, j): random.randint(5, 15) if i != j else 0
    for i in range(len(nodes)) for j in range(len(nodes))
}

# Parameters
NUM_DAYS = 3
MAX_TIME_PER_DAY = 40
POP_SIZE = 10
NUM_GENERATIONS = 20
EPSILON = 0.05
NUM_ALTERNATIVES = 3
JACCARD_THRESHOLD = 0.3

# Helper functions
def is_day_feasible(day_route: List[int]) -> bool:
    time = 0
    current_node = 0
    for node in day_route:
        time += travel_times[(current_node, node)]
        if time < nodes[node].time_window[0]:
            time = nodes[node].time_window[0]
        elif time > nodes[node].time_window[1]:
            return False
        current_node = node
    time += travel_times[(current_node, 0)]
    return time <= MAX_TIME_PER_DAY

def is_multi_day_feasible(route_by_day: List[List[int]]) -> bool:
    return all(is_day_feasible(day) for day in route_by_day)

def compute_profit(route_by_day: List[List[int]]) -> int:
    return sum(nodes[i].profit for day in route_by_day for i in day)

def jaccard_similarity_multi(route1: List[List[int]], route2: List[List[int]]) -> float:
    flat1 = set(i for day in route1 for i in day)
    flat2 = set(i for day in route2 for i in day)
    all_nodes_set = set(range(len(nodes)))
    binary_flat1 = [1 if node in flat1 else 0 for node in all_nodes_set]
    binary_flat2 = [1 if node in flat2 else 0 for node in all_nodes_set]
    return jaccard_score(binary_flat1, binary_flat2)


def generate_initial_population() -> List[List[List[int]]]:
    population = []
    all_nodes = list(range(1, len(nodes)))
    while len(population) < POP_SIZE:
        random.shuffle(all_nodes)
        split = np.array_split(all_nodes, NUM_DAYS)
        candidate = [list(day) for day in split if is_day_feasible(list(day))]
        if len(candidate) == NUM_DAYS:
            population.append(candidate)
    return population

def crossover(parent1: List[List[int]], parent2: List[List[int]]) -> List[List[int]]:
    child = []
    for d in range(NUM_DAYS):
        combined = list(set(parent1[d] + parent2[d]))
        random.shuffle(combined)
        sub_day = [n for n in combined if n not in itertools.chain(*child)]
        child.append(sub_day[:len(sub_day)//2])
    return child

def mutate(route: List[List[int]]) -> List[List[int]]:
    day_from, day_to = random.sample(range(NUM_DAYS), 2)
    if route[day_from]:
        node = random.choice(route[day_from])
        route[day_from].remove(node)
        if node not in route[day_to]:
            route[day_to].append(node)
    return route

def genetic_algorithm(base_route: List[List[int]], base_profit: int, existing_routes: List[List[List[int]]]) -> List[List[int]]:
    population = generate_initial_population()
    best_candidate = base_route
    for _ in range(NUM_GENERATIONS):
        new_population = []
        for _ in range(len(population)):
            p1, p2 = random.sample(population, 2)
            child = crossover(p1, p2)
            child = mutate(child)
            if is_multi_day_feasible(child):
                profit = compute_profit(child)
                if profit >= (1 - EPSILON) * base_profit:
                    if all(jaccard_similarity_multi(child, r) >= JACCARD_THRESHOLD for r in existing_routes):
                        new_population.append(child)
                        if profit > compute_profit(best_candidate):
                            best_candidate = child
        if not new_population:
            break
        population = new_population
    return best_candidate

# Main MGA loop
base_route = max(generate_initial_population(), key=compute_profit)
base_profit = compute_profit(base_route)
alternatives = [base_route]
routes_set = [base_route]

while len(alternatives) < NUM_ALTERNATIVES:
    alt = genetic_algorithm(base_route, base_profit, routes_set)
    if alt and alt not in routes_set:
        alternatives.append(alt)
        routes_set.append(alt)
    else:
        break

# Output results
for i, r in enumerate(alternatives):
    flat = [n for day in r for n in day]
    print(f"Alternative {i+1}:\n  Total Profit = {compute_profit(r)}\n  Route:")
    for d, day in enumerate(r):
        print(f"    Day {d+1}: 0 -> {' -> '.join(map(str, day))} -> 0")
    if i > 0:
        print(f"  Jaccard w/ Base = {jaccard_similarity_multi(r, base_route):.2f}\n")

Alternative 1:
  Total Profit = 88
  Route:
    Day 1: 0 -> 3 -> 2 -> 0
    Day 2: 0 -> 1 -> 4 -> 0
    Day 3: 0 -> 5 -> 0
